This notebooks is an attempt to infere a home address based on twitter data from an user

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data/MrNaassir.csv')

In [3]:
data.head()

,Unnamed: 0,Unnamed: 0.1,UserID,Username,Tweeted_At,Tweet ID,Location,Geo,Place,Profile_Create_at,Retweeted_Count,Text,geometry,datetime
0,48249,52990,NaN,MrNaassir,Mon Feb 09 22:03:05 +0000 2015,564907364563025922,New Yark City.New York. U.S.A.,"{u'type': u'Point', u'coordinates': [40.690623...","{u'full_name': u'Queens, NY', u'url': u'https:...",Sat Jun 20 00:18:35 +0000 2009,0,@brookejune9 http://t.co/Wfs1gQQZ1K,POINT (40.6906234 -73.82367859999999),2015-02-09 22:03:05
1,48389,53130,NaN,MrNaassir,Mon Feb 09 21:58:41 +0000 2015,564906255731658753,New Yark City.New York. U.S.A.,"{u'type': u'Point', u'coordinates': [40.690586...","{u'full_name': u'Queens, NY', u'url': u'https:...",Sat Jun 20 00:18:35 +0000 2009,1,@b13436527cee4e9:@nsmo2 @Queen_manal @KmQlJjrq...,POINT (40.6905861 -73.82390989),2015-02-09 21:58:41
2,48562,53303,NaN,MrNaassir,Mon Feb 09 21:52:43 +0000 2015,564904752870600704,New Yark City.New York. U.S.A.,"{u'type': u'Point', u'coordinates': [40.690459...","{u'full_name': u'Queens, NY', u'url': u'https:...",Sat Jun 20 00:18:35 +0000 2009,0,@AfredoMM1965: @MrNaassir merci beaucoup ! \n\...,POINT (40.6904596 -73.82384273),2015-02-09 21:52:43
3,48583,53324,NaN,MrNaassir,Mon Feb 09 21:51:55 +0000 2015,564904552747773952,New Yark City.New York. U.S.A.,"{u'type': u'Point', u'coordinates': [40.690524...","{u'full_name': u'Queens, NY', u'url': u'https:...",Sat Jun 20 00:18:35 +0000 2009,1,@almohid @dermott_ann @GCHDutta @Domi52778 @Le...,POINT (40.69052428 -73.82379426999999),2015-02-09 21:51:55
4,48589,53330,NaN,MrNaassir,Mon Feb 09 21:51:34 +0000 2015,564904465212669952,New Yark City.New York. U.S.A.,"{u'type': u'Point', u'coordinates': [40.690524...","{u'full_name': u'Queens, NY', u'url': u'https:...",Sat Jun 20 00:18:35 +0000 2009,1,@royca60 @almohid @Leonor188 @dermott_ann @GCH...,POINT (40.69052428 -73.82379426999999),2015-02-09 21:51:34
